In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import libraries
import gzip
import numpy as np
import pandas as pd
import string

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
# define X, y and test
X = train['excerpt']
y = train['target']
test_text = test["excerpt"]

In [ ]:
# lower the text
X = X.str.lower()
test_text = test_text.str.lower()

In [ ]:
# def remove_punctuations(text):
#     for punctuation in string.punctuation:
#         text = text.replace(punctuation, '')
#     return text

# X = X.apply(remove_punctuations)
# test_text = test_text.apply(remove_punctuations)


In [ ]:
# # apply Porter Stemmer
# from nltk.stem import PorterStemmer
# ps = PorterStemmer()

# def stem_sentences(sentence):
#     tokens = sentence.split()
#     stemmed_tokens = [ps.stem(token) for token in tokens]
#     return ' '.join(stemmed_tokens)

# X = X.apply(stem_sentences)
# test_text = test_text.apply(stem_sentences)

In [ ]:
# # lemmatize the text
# from nltk.stem import WordNetLemmatizer
# import nltk
# # nltk.download('wordnet')

# lemmatizer = WordNetLemmatizer()
# def lemmatize_sentences(sentence):
#     tokens = sentence.split()
#     stemmed_tokens = [lemmatizer.lemmatize(token) for token in tokens]
#     return ' '.join(stemmed_tokens)

# X = X.apply(lemmatize_sentences)
# test_text = test_text.apply(lemmatize_sentences)

In [ ]:
import nltk
# nltk.download('stopwords')

from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

X = X.apply(lambda text: remove_stopwords(text))
test_text = test_text.apply(lambda text: remove_stopwords(text))

In [ ]:
# from collections import Counter
# cnt = Counter()
# for text in X.tolist() + test_text.tolist():
#     for word in text.split():
#         cnt[word] += 1
        
# most_common_words = [x[0] for x in cnt.most_common(100)]

# def remove_most_common_words(text):
#     return " ".join([word for word in str(text).split() if word not in most_common_words])

# X = X.apply(lambda text: remove_most_common_words(text))
# test_text = test_text.apply(lambda text: remove_most_common_words(text))

In [ ]:
X_new = []
for index, value in X.items():
    X_new.append(value)
    
y_new = []
for index, value in y.items():
    y_new.append(value)
    
real_test = []
for index, value in test_text.items():
    real_test.append(value)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.20)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

In [ ]:
# m = RandomForestRegressor()
# vectorizer = CountVectorizer(max_features=5000, ngram_range=(1, 2))
# m.fit(vectorizer.fit_transform(X_new + real_test)[:len(X_new)], y_new)

In [ ]:
EMBEDDING_FILE = '../input/glovefile/glove.6B.100d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    embeddings[word] = embd

In [ ]:
def get_feature_vectors(sentence):
    words = sentence.split()
    feature_vec = np.zeros((100,),dtype="float32")
    i=0
    for word in words:
        try:
            feature_vec = np.add(feature_vec, embeddings.get(word))
        except:
            i = i + 1
    if len(words) > 0:
        feature_vec = np.divide(feature_vec, len(words)- i)
    return feature_vec

In [ ]:
train_vectors = np.array([get_feature_vectors(sentence) for sentence in X_train])
test_vectors = np.array([get_feature_vectors(sentence) for sentence in X_test])
real_test_vectors = np.array([get_feature_vectors(sentence) for sentence in real_test])
y_full = np.array(y_new)
train_vectors_full = np.array([get_feature_vectors(sentence) for sentence in X_new])

In [ ]:
# m.fit(train_vectors_full, y_new)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold

In [ ]:
def create_model():
    
    model = keras.Sequential([
        layers.Dense(units=512, kernel_initializer='normal', activation='linear', input_shape=[100]),
        layers.Dense(units=256, kernel_initializer='normal', activation='linear'),
        layers.Dense(units=128, kernel_initializer='normal', activation='linear'),
        layers.Dropout(0.25),
        # the linear output layer 
        layers.Dense(units=1, kernel_initializer='normal', activation='linear'),
    ])
    
    model.compile(optimizer = 'adam', loss='mean_squared_error')
    
    return model

In [ ]:
model = create_model()
model_KR = KerasRegressor(build_fn = create_model, batch_size = 16, epochs = 50)
kfolds = cross_val_score(model_KR, train_vectors_full, y_full, cv = 10)
print(kfolds.mean())
print('The mean accuracy:', kfolds.mean())


In [ ]:
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("", monitor="val_loss", verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='auto', restore_best_weights=True)

history = model_KR.fit(
    train_vectors_full, y_full,
    validation_split=0.4,
    batch_size=16,
    epochs=50,
    callbacks = [early_stop, checkpoint, reduce_lr]
)


In [ ]:
# predict 
# pred_test = m.predict(real_test_vectors)
pred_test = model_KR.predict(real_test_vectors)
pred_test_list = [i for i in pred_test]
pred_test_list

In [ ]:
submission = pd.DataFrame({'id' : test['id'], 'target' : pred_test_list})
submission.to_csv('/kaggle/working/submission.csv', index=False)
submission.head(7)